## Бонусное ДЗ №4 Нейросетевое моделирование следующей корзины

#### В этой домашке вам предстоит реализовать нейросеть, которая моделирует паттерны повторных покупок людей

#### Дата выдачи: 11.06.25

#### Жесткий дедлайн: 18.06.25 23:59 MSK

<script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/latest.js?config=TeX-AMS-MML_HTMLorMML"></script>

## Описание задания

В этом задании вам предстоит реализовать нейронную сеть для моделирования поведения пользователей в доменах с высокой частотой повторяющихся взаимодействий между пользователями и товарами (айтемами).

### Структура задания

Задание состоит из трёх основных блоков:

1. **Подготовка данных и базовый подход (макс 3 балла)**
   - Подготовка датасета для экспериментов
   - Реализация простого базового решения (TopPersonal) для последующего сравнения

2. **Реализация нейронной сети BasketHSERec (макс 4 балла)**
   - Реализация архитектуры, моделирующей динамику взаимодействий пользователя с каждым конкретным товаром
   - Учёт временных закономерностей в истории покупок

3. **Эксперименты и анализ (макс 3 балла)**
   - Проведение сравнительных экспериментов между предложенной моделью и базовым подходом
   - Анализ результатов и формулирование выводов

### Особенности задачи

Задание фокусируется на сценариях с:
- Высокой повторяемостью покупок (например, продукты питания, бытовая химия)
- Важностью временных паттернов в поведении пользователей
- Необходимостью персонализированных рекомендаций на основе истории взаимодействий

# Описание модели BasketHSERec


Авторам задания не известна научная статья, в которой бы точно описывалась предлагаемая архитектура, однако она частично вдохновлена упрощёнными идеями из следующих работ:
1. [ReCaNet](https://irlab.science.uva.nl/wp-content/papercite-data/pdf/ariannezhad-2022-recanet.pdf)
2. Недавняя статья [SAFeRec](https://arxiv.org/pdf/2412.14302) от авторов курса. На самом деле, BasketHSERec - это SAFeRec, но без трансформерной части, чтобы не усложнять реализацию

## Постановка задачи

Пусть имеется:
- Множество пользователей $u \in U$
- Множество товаров (айтемов) $i \in I$

Для каждого пользователя составим матрицу $W^u \in \mathbb{R}^{T \times |I|}$, где $w_t^u$ - multihot-вектор корзины покупок в момент времени $t$. 

Здесь:
- $t$ представляет дискретные периоды времени от 1 до $T$ (например, дни)
- $w_{t,i}^u = 1$, если пользователь купил товар $i$ в момент $t$, и $0$ в противном случае

**Важно**: матрица будет очень разреженной, поэтому необходимо использовать структуры данных для работы с разреженными матрицами. Но, можно попробовать и на full-connected слоях

Задача: предсказать следующую корзину покупок пользователя в момент времени $T+1$ (например, на следующий день). 

**Примечание**: при валидации на разных временных отрезках значение $T$ будет разным для разных пользователей.


## Базовый подход (TopPersonal)


Для пользователя $u$ с историей покупок  $W^u \in \mathbb{R}^{T \times |I|}$:

1. **Суммарный вектор покупок**:
   $$
   s^u = \sum_{t=1}^T w_t^u \quad \in \mathbb{R}^{|I|}
   $$
   где $s_i^u$ — сколько раз пользователь $u$ покупал товар $i$.

2. **Добавление популярности товаров**:
   $$
   p_u^{\text{TopPersonal}} = s^u + pop_{i} \quad \in \mathbb{R}^{|I|}
   $$
   Здесь $\text{pop}_i$ - нормированная популярность товара

3. **Рекомендация**:
   - Сортируем товары по убыванию $ p_u^{\text{TopPersonal}} $
   - Топ-$k$ товаров — предсказанная корзина на $T+1$

## Мотивация для BasketHSERec

Рассмотрим вектор $h_u^i \in \mathbb{R}^T$ - строку в матрице $W^u$, соответствующую товару $i$. Если просто просуммировать его элементы, мы получим общее количество покупок товара, но не учтём временные закономерности.

**Пример**: Допустим, мы наблюдаем 10 дней, и пользователь купил кефир в 1-й, 3-й и 7-й дни. Тогда:

$$h_u^i = [1, 0, 1, 0, 0, 0, 1, 0, 0, 0]$$

Простое суммирование частот не учитывает периодичность покупок. Рассмотрим два случая:
1. $[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]$
2. $[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]$

В первом случае товар был куплен 10 дней назад, во втором - "вчера". Вероятность повторной покупки во втором случае выше, хотя это также зависит от паттернов покупок товара.

## Архитектура BasketHSERec

Определим модель как:

$$y_u^i = \text{BasketHSERec}(h_u^i) = f(h_u^i, i) \in \mathbb{R}$$

где $y_u^i$ предсказывает вероятность покупки товара $i$ пользователем $u$ в момент времени $T+1$.

### Реализация:
1. Функция $f(h_u^i, i)$:
   - Конкатенируем вектор $h_u^i$ и эмбеддинг товара $i$
   - Пропускаем через несколько полносвязных (FC) слоёв с нелинейной активацией
   - На выходе получаем одно число (вероятность покупки)

2. Функция потерь:
   - Используем бинарную кросс-энтропию для классификации (покупка/нет покупки)
   - **Важно**: рассматриваем только товары, которые пользователь уже покупал ранее


## Описание данных

Для выполнения задания используется датасет **Dunnhumby - The Complete Journey**, содержащий историю покупок пользователей. 

**Исходные данные**:  
[Официальная версия на Kaggle](https://www.kaggle.com/datasets/frtgnn/dunnhumby-the-complete-journey)  

**Подготовленная версия**:  
[Яндекс.Диск](https://disk.yandex.ru/d/PPpfc9a4I9Y38w) (удалены лишние колонки для удобства работы)

## Система оценивания

### 1. Подготовка данных (1 балл)
- [ ] Подготовлены обучающий и тестовый датасеты
- [ ] Использована схема разбиения:
  - Leave-one-basket (последняя корзина как тест) **или**
  - Time-based split (последний день как тест)
- [ ] *(Опционально)* Реализована валидационная выборка и перебор параметров

### 2. Реализация TopPersonal (2 балла)
- [ ] Корректно реализован алгоритм TopPersonal
- [ ] Рассчитаны метрики качества:
  - NDCG@3 и Recall@3
  - NDCG@10 и Recall@10
- [ ] Результаты представлены в понятном формате

### 3. Обучение BasketHSERec (2 балла)
- [ ] Реализован код обучения модели
- [ ] Продемонстрирован процесс обучения:
  - Графики обучения/потерь
  - Логи процесса (например, через tqdm)
- [ ] Сохранены веса обученной модели

### 4. Сравнение с TopPersonal (3 балла)
- [ ] Проведено сравнение BasketHSERec и TopPersonal
- [ ] Показано преимущество BasketHSERec по метрикам:
  - NDCG@3/10
  - Recall@3/10
- [ ] Результаты представлены в табличном виде с четкими выводами

### 5. Анализ архитектурных решений (2 балла)
- [ ] Проведено сравнение вариантов BasketHSERec:
  
  | Вариант               | NDCG@3 | Recall@3 | NDCG@10 | Recall@10 |
  |-----------------------|--------|----------|---------|-----------|
  | a) History + Embeddings |        |          |         |           |
  | b) Только Embeddings    |        |          |         |           |
  | c) Только History       |        |          |         |           |
  | TopPersonal (baseline)  |        |          |         |           |

- [ ] Сделаны содержательные выводы о влиянии компонентов модели

In [12]:
import pandas as pd

df = pd.read_parquet('dunnhumby_norm.pq')
df.head()

,user_id,day,item_id
0,2375,1,1004906
1,2375,1,1033142
2,2375,1,1036325
3,2375,1,1082185
4,2375,1,8160430


### 1. Подготовка данных (1 балл)


### 2. Реализация TopPersonal (2 балла)


### 3. Обучение BasketHSERec (2 балла)

### 4. Сравнение с TopPersonal (3 балла)


### 5. Анализ архитектурных решений (2 балла)